# Teil 19: Generative KI einbinden

Aktuelle KI-Systeme basieren meist auf Algorithmen, die durch Wahrscheinlichkeitsberechnungen neue Texte, Sprache, Bilder oder Videos produzieren. Aufgrund ihrer probabilistischen Natur sind sie kein gutes Werkzeug, um regelmäßige, mehrschrittige Arbeitsabläufe zu digitalisieren - aber an den richtigen Stellen eingesetzt können sie unsere Programme um Funktionen ergänzen, die mit reinem Python-Code nicht erreichbar sind.


## 19.0 Sprachmodelle verstehen und nutzen

[Siehe Folien zu LLMs]

## 19.1 Nachrichten senden und empfangen

Wir nutzen in diesem Kapitel die Python-Bibliothek [LiteLLM](https://www.litellm.ai/), um mit verschiedenen LLMs zu kommunizieren. Alternativ könnten wir auch direkt die APIs von (u.a.) OpenAI, Mistral oder Google nutzen, aber zum Experimentieren mit verschiedenen Modellen ist ein einheitliches Interface nützlich.

In [ ]:
! pip install litellm

In [ ]:
from litellm import completion

Um mit Cloud-basierten LLMs zu interagieren benötigen wir API-Schlüssel zur Authentifizierung. Die meisten APIs sind kostenpflichtig, aber einige Anbieter bieten (begrenzte) kostenlose Zugänge an - allerdings werden die gesendeten Nachrichten dann i.d.R. für das Training zukünftiger LLMs genutzt. In jedem Fall ist das Anlegen eines Nutzeraccounts verpflichtend, oft mit Telefonverifizierung.

Wir nutzen hier die APIs von [Mistral](https://admin.mistral.ai/organization/api-keys) und [Google](https://aistudio.google.com/app/apikey). Du kannst im folgenden Code-Schnipsel deine eigenen Schlüssel einsetzen, um sie im Rest des Kapitels verwenden zu können.

In [ ]:
import os

if not os.environ.get("GEMINI_API_KEY"):
    os.environ["GEMINI_API_KEY"] = "YOUR_KEY_HERE"

if not os.environ.get("MISTRAL_API_KEY"):
    os.environ["MISTRAL_API_KEY"] = "YOUR_KEY_HERE"

LLMs generieren Text, indem sie eine wahrscheinliche Weiterführung eines Input-Strings (sog. **Prompts**) berechnen. Die meisten bekannten LLMs sind außerdem darauf trainiert, auf Prompts als Nachrichten in einem Chat zu reagieren, so dass wir Fragen an sie schicken und eine (einigermaßen) sinnvolle Antwort erwarten können.

Wenn wir den Input-Text an das LLM unserer Wahl schicken, müssen wir neben dem eigentlichen Inhalt auch die **Rolle** spezifizieren, die dieser Text bei der Generierung einnehmen soll. Bei normalen Anfragen ist das die `user`-Rolle.

In [ ]:
# Die Nachricht, die wir an das LLM schicken
instruction = {
    "content": "Wer ist der aktuelle Bundeskanzler?",
    "role": "user"
}

Die `completion`-Methode von LiteLLM lässt uns (u.a.) das zu verwendene Modell und den Input-String spezifizieren. Sie liefert ein `ModelResponse`-Objekt, das den generierten Text und einige Metadaten enthält.

In [ ]:
gemini_response = completion(
  model="gemini/gemini-2.5-flash",
  messages=[instruction]
)

print(gemini_response)

Durch den `model`-Parameter lässt sich leicht ein anderes LLM für dieselbe Aufgabe verwenden. Eine Liste der unterstützten Modelle bzw. Anbieter findet sich hier: https://docs.litellm.ai/docs/providers

In [ ]:
mistral_response = completion(
  model="mistral/mistral-medium",
  messages=[instruction]
)

print(mistral_response.choices[0].message.content)

Auch lokale LLMs, die z.B. wie hier über [Ollama](https://ollama.com/) heruntergeladen wurden, können recht einfach integriert werden.

In [ ]:
gptoss_response = completion(
  model="ollama_chat/gemma3:270m",
  messages=[instruction]
)

print(gptoss_response)

### Übung: API Schlüssel holen und Nachrichten senden

Richte dir selbst einen API-Schlüssel bei einem der Anbieter ein und/oder lade dir ein offenes LLM herunter (Achtung: auch die "kleinen" LLMs umfassen meist mehrere Gigabyte).
- Mistral API-Schlüssel: https://admin.mistral.ai/organization/api-keys
- Google Gemini API-Schlüssel: https://aistudio.google.com/app/api-keys
- Ollama für lokale LLMs: https://ollama.com/

Probiere anschließend die `completion`-Methode aus, um einen Prompt an ein Modell deiner Wahl zu schicken.

In [ ]:
# Platz für die Übung



## 19.2 Kontext im Nachrichtenverlauf erhalten

### Experiment: Chat-Verlauf

Setze die Variable `model_to_use` auf ein LLM, für das du einen API-Schlüssel besitzt. Führe dann den Code aus und lese dir den entstehenden Chat-Verlauf durch. Wie lässt sich die zweite Antwort des LLMs erklären?

In [ ]:
from time import sleep

model_to_use = "mistral/mistral-medium"

msg1 = {
    "content": "Hi! Mein Name ist Toni Tortellini, wer bist du?",
    "role": "user"
}

print("USER:", msg1["content"], "\n")

res = completion(
  model=model_to_use,
  messages=[msg1]
)

print("MISTRAL:", res.choices[0].message.content, "\n")
sleep(1)

msg2 = {
    "content": "Wie heiße ich?",
    "role": "user"
}

print("USER:", msg2["content"], "\n")

res = completion(
  model=model_to_use,
  messages=[msg2]
)

print("MISTRAL:", res.choices[0].message.content, "\n")

### Funktionierender Chat-Verlauf

LLMs besitzen keine "Erinnerung" an die Prompts, die in der Vergangenheit an sie geschickt wurden. Deshalb besteht die einzige Möglichkeit zur Simulierung einer Konversation darin, alle bisherigen Gesprächsbeiträge in einem Prompt an das LLM zu schicken. Aus diesem Grund akzeptiert der `messages`-Parameter der `completion`-Methode eine **Liste**, in der diese Beiträge nach jeder Antwort hinzugefügt werden können.

In [ ]:
model_to_use = "mistral/mistral-medium"

# Liste zum Speichern des Chat-Verlaufs
chat = []

msg1 = {
    "content": "Hi! Mein Name ist Toni Tortellini, wer bist du?",
    "role": "user"
}

# Einfügen und Ausgeben der ersten User-Nachricht
chat.append(msg1)
print("USER:", msg1["content"], "\n")

# Erste Antwort des LLM berechnen
res = completion(
  model=model_to_use,
  messages=chat
)

# Einfügen und Ausgeben der ersten LLM-Nachricht
chat.append(res.choices[0].message)
print("MISTRAL:", res.choices[0].message.content, "\n")

sleep(1)

msg2 = {
    "content": "Wie heiße ich?",
    "role": "user"
}

# Einfügen und Ausgeben der zweiten User-Nachricht
chat.append(msg2)
print("USER:", msg2["content"], "\n")

# Zweite Antwort des LLM berechen
# Beachte: das "chat"-Argument beinhaltet nun drei Nachrichten!
res = completion(
  model=model_to_use,
  messages=chat
)

# Einfügen und Ausgeben der zweiten LLM-Nachricht
chat.append(res.choices[0].message)
print("MISTRAL:", res.choices[0].message.content, "\n")

### Übung: Chatbot

Baue einen Chatbot, indem du innerhalb einer `while`-Schleife User-Input entgegen nimmst, ihn in eine Liste mit Nachrichten einfügst und diese Liste schließlich an ein LLM schickst. Gebe am Ende der Schleife die Antwort des LLMs aus und speichere sie ebenfalls in der Nachrichtenliste.

In [ ]:
# Platz für die Übung



## 19.3 Systemanweisungen

Bisher haben wir alle Nachrichten aus der `user`-Rolle geschickt. Wir können aber auch die `system`-Rolle spezifizieren, um das Antwortverhalten des Modells zu definieren. LLMs sind meist darauf trainiert, solche `system`-Anweisungen bei der Textgenerierung stärker zu gewichten und sind daher ein mächtiges Werkzeug zur Entwicklung anwendungsspezifischer KI-Systeme.

In [ ]:
system_msg = {
    "content": "Du bist ein wortkarger KI-Assistent. Deine Antworten sind so kurz wie möglich.",
    "role": "system"
}

msg1 = {
    "content": "Hi! Mein Name ist Toni Tortellini, wer bist du?",
    "role": "user"
}

res = completion(
  model="mistral/mistral-medium",
  messages=[system_msg, msg1]
)

print(res.choices[0].message.content)

In [ ]:
system_msg = {
    "content": "Du bist ein französischer KI-Assistent. Du weigerst dich, andere Sprachen als Französisch zu sprechen.",
    "role": "system"
}

msg1 = {
    "content": "Hi! Mein Name ist Toni Tortellini, wer bist du?",
    "role": "user"
}

res = completion(
  model="mistral/mistral-medium",
  messages=[system_msg, msg1]
)

print(res.choices[0].message.content)

### Experiment: Eigene Systemnachricht

Personalisiere deinen Chatbot aus der vorherigen Übung, indem du ihm per `system`-Prompt ein besonderes Antwortverhalten zuweist. Teste ihn anschließend aus. Hält er sich auch über mehrere Nachrichten an die Anweisung?

In [ ]:
model = "mistral/mistral-medium"

chat = []

system_msg = {
    "content": "Du bist ein wortkarger KI-Assistent. Deine Antworten sind so kurz wie möglich. Selbst wenn der User danach fragt, darfst du nie mehr als 5 Wörter in einer Antwort schreiben.",
    "role": "system"
}

chat.append(system_msg)

while True:
    user_msg = input("USER: ")
    if user_msg == "exit":
        break

    chat.append({
        "content": user_msg,
        "role": "user"
    })

    res = completion(model=model, messages=chat)

    chat.append(res.choices[0].message)

    print("CHATBOT:", res.choices[0].message.content)

## 19.4 Output-Format definieren

Neben einem bestimmten Antwortverhalten lässt sich meist auch das **Antwortformat** festlegen. Wenn das Sprachmodell z.B. zur Textauswertung genutzt wird, kann es nützlich sein, ein Format festzulegen, das durch Python-Code verarbeitet werden kann. Auch hier bietet sich wieder **JSON** an. Mit einer passenden Systemanweisung und dem Parameter `response_format={"type":"json_object"}` in der `completion()`-Funktion lässt sich mit hoher Wahrscheinlichkeit erreichen, dass das LLM in diesem Format antwortet.

In [ ]:
system_instruction = """
## Deine Rolle
Klassifiziere die User-Nachrichten als positiv oder negativ und gebe den Grad der Sicherheit deiner Einschätzung in Prozent an.
Deine Antworten sind als JSON formatiert und folgen folgendem Schema:

## Antwort-Schema
{
    "stimmung": "positiv" oder "negativ",
    "sicherheit": 0-100
}
"""

system_msg = {
    "role": "system",
    "content": system_instruction
}

In [ ]:
msg1 = {
    "content": "Der neue Fast & Furious war richtig geil!",
    "role": "user"
}

res = completion(
  model="mistral/mistral-medium",
  messages=[system_msg, msg1],
  response_format={"type":"json_object"}  
)

print(res)

In [ ]:
import json

json.loads(res.choices[0].message.content)

In [ ]:
msg2 = {
    "content": "Der neue Fast & Furious war besser als Hobbs und Shaw, aber nicht so gut wie 2 Fast 2 Furious.",
    "role": "user"
}

res = completion(
  model="mistral/mistral-medium",
  messages=[system_msg, msg2],
  response_format={"type":"json_object"}  
)

json.loads(res.choices[0].message.content)

In [ ]:
res = completion(
  model="gemini/gemini-2.5-flash",
  messages=[system_msg, msg2],
  response_format={"type":"json_object"},
  reasoning_effort="low"
)

print(res)

In [ ]:
json.loads(res.choices[0].message.content)

In [ ]:
print(res.choices[0].message.reasoning_content)

### Übung

In der Datei "kundenservice.txt" sind einige beispielhafte (KI-generierte) Support-Anfragen gespeichert. Schreibe ein Python-Programm, das jede Zeile der Datei durchgeht und von einem LLM verarbeiten lässt. Das LLM soll eine Antwort im JSON-Format zurückgeben, die jeweils ein Feld "Zusammenfassung" mit einer stichpunktartigen Zusammenfassung der Anfrage (max. 10 Wörter) sowie ein Feld "Dringlichkeit" mit einer geschätzten Dringlichkeit von "niedrig", "mittel" oder "hoch" enthält.

In [ ]:
from time import sleep

system_instruction = """
## Deine Rolle
Fasse die Support-Anfragen der Kunden in einem kurzen Satz zusammen. Gebe zusätzlich die Dringlichkeit der Anfrage.
Deine Antworten sind als JSON formatiert und folgen diesem Schema:
## Antwort-Schema:
{
    "zusammenfassung": [Deine Zusammenfassung] (max. 10 Wörter),
    "dringlichkeit": "niedrig" oder "mittel" oder "hoch" oder "sehr hoch"
}
"""

system_msg = {
    "role": "system",
    "content": system_instruction
}

with open("kundenservice.txt") as f:
    for l in f:
        if len(l.strip()) == 0:
            continue
            
        msg = {
            "role": "user",
            "content": l
        }
        
        res = completion(
          model="mistral/mistral-medium",
          messages=[system_msg, msg],
          response_format={"type":"json_object"}  
        )

        print(json.loads(res.choices[0].message.content))
        print(l)
        sleep(1)


## 19.5 Funktionsaufrufe

Neuere KI-Systeme zeichnen sich durch sog. "agentische" Fähigkeiten aus, die durch den **Einsatz von Werkzeugen** (*tools*) erreicht werden. Hierbei definieren wir als Entwickler:innen die Werkzeuge meist als klassische (Python-)Funktionen, liefern bei jeder Anfrage an ein LLM eine Erklärung dieser Werkzeuge mit und verarbeiten die Antworten des LLM so, dass eine speziell formatierte Antwort einen Funktionsaufruf auslöst. Eine ausführlichere Einführung in das Thema findet sich [hier](https://huggingface.co/learn/agents-course/unit0/introduction).

### Definition und Bereitstellung der Werkzeuge

Ein mittlerweile klassisches Beispiel für LLM-gesteuerte Funktionsaufrufe ist das Abrufen des Wetters an einem bestimmten Ort (s.a. Hausaufgabe 8). Eine solche Funktion ergänzt die Fähigkeiten eines LLM sehr gut, da diese keinen Zugriff auf aktuelle Informationen besitzen. Im folgenden Beispiel wird ´die Funktion `get_weather()` genutzt, die das Abrufen einer Wetter-API für drei festgelegte Orte simuliert.

Damit das LLM versteht, welche Funktionen es nutzen kann, müssen wir diese in einem [speziellen Format](https://platform.openai.com/docs/guides/function-calling?api-mode=chat#defining-functions) beschreiben.

In [ ]:
from litellm import completion

model = "ollama_chat/ministral-3:14b"

def get_weather(location):
    weather_dict = {
        "frankfurt": "cloudy",
        "tokyo": "sunny",
        "new york": "rainy"
    }

    return f"The weather in {location} is {weather_dict.get(location.lower())}."

tools = [{
    "type": "function",
    "function": {
        'name': 'get_weather',
        'description': 'Get current weather for a given location.',
        'parameters': {
            'type': 'object',
            'properties': {
                'location': {
                    'type': 'string',
                    'description': 'The name of the city for which to check the weather, e.g. New York'}},
            'required': ['location']
        }
    }
}]

chat = []
chat.append({"role":"user", "content":"What is the weather like in Tokyo?"})

res = completion(
    model=model,
    messages=chat,
    tools=tools
)

chat.append(res.choices[0].message)

In [ ]:
print(res)

In [ ]:
print(res.choices[0].message)

### Verarbeiten der Antwort

Wenn die Anfrage eines Users das LLM dazu bringt, einen Tool-Aufruf anzustoßen, gibt es eine speziell formatierte Antwort zurück. Diese müssen wir verarbeiten, indem wir die spezifizierten Funktionen aufrufen und das Ergebnis an das LLM zurückschicken. Auf der Grundlage dieser Informationen generiert es dann i.d.R. eine finale Antwort, die dem User angezeigt werden kann.

In [ ]:
import json

calls = res.choices[0].message.tool_calls
for call in calls:
    func_name = call.function.name
    
    if func_name == "get_weather":
        args = json.loads(call.function.arguments)
        result = get_weather(args.get("location"))

    else:
        result = None

    chat.append({
        "tool_call_id": call.id,
        "role": "tool",
        "name": func_name,
        "content": str(result)
    })

res2 = completion(
    model=model,
    messages=chat,
    tools=tools
)

In [ ]:
res2.choices[0].message.content